This file includes the super-family probabilities from the v12.8 classifier used to filter the slides, hopefully resulting in an even larger training set. 

In [1]:
import numpy as np
import sklearn
import pandas as pd
import h5py

import matplotlib.pyplot as plt

from skimage.transform import resize, pyramid_gaussian
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

import zarr

import os
import sys
import re

import openslide

from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay



In [9]:
slide_meta = pd.read_csv("../metadata/labels_with_new_batch_v12.8.csv")
ct_scoring = pd.read_csv("../metadata/CT_3_Class_Draft.csv")



ct_scoring["txt_idat"] = ct_scoring["idat"].astype("str")
ct_scoring.index = ct_scoring.txt_idat
slide_meta.index = slide_meta.idat
ct_scoring = ct_scoring.drop("txt_idat", axis=1)
slide_meta = slide_meta.drop("idat", axis=1)
slide_annots = slide_meta.join(ct_scoring, lsuffix="l")


myx = [x in ["Chromothripsis", "No Chromothripsis"] for x in slide_annots.CT_class]

slide_annots = slide_annots.loc[myx]
slide_names = slide_annots.uuid

# slide_names
slide_annots.CT_class


oncotree_map = pd.read_csv("../metadata/MappingClassifierToOncotree.csv")

slide_annots = slide_annots.merge(oncotree_map, left_on="max_super_family_class", right_on="Super Family", how="left")

slide_annots = slide_annots.loc[slide_annots['Oncotree code'].isin(['DIFG'])]
slide_annots = slide_annots.loc[slide_annots.maxscore_superfamily * 100 >= 80]

slide_annots

,Unnamed: 0l,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,...,maxscore_family,prediction_class,maxscore_class,prediction_subclass,maxscore_subclass,Unnamed: 0,idat,CT_class,Super Family,Oncotree code
4,14,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,74022,"supratentoriell, frontal rechts",11.0,5459,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,0.999999,dysembryoplastic neuroepithelial tumour,0.999795,Dysembryoplastic neuroepithelial tumour,0.999795,76.0,10003886259_R02C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
8,40,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,73136,frontal left,24.0,5311,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade",...,0.988608,"diffuse glioma, IDH-mutant and 1p19q retained ...",0.947667,"Astrocytoma, IDH-mutant; high grade",0.947667,494.0,10006823123_R01C01,Chromothripsis,Adult-type diffuse gliomas,DIFG
10,47,F488896F-A788-4EAA-9B34-DB063CD29189,73178,"supratentoriell, temporal links",8.0,5342,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,0.999970,dysembryoplastic neuroepithelial tumour,0.987715,Dysembryoplastic neuroepithelial tumour,0.987715,510.0,10006823130_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
15,66,B0250F69-2019-435C-8C95-A59B2A8CB70F,74226,"supratentoriell, frontal rechts",64.0,5520,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade ganglioglial/neuroepithelial tumour,supratentorial pilocytic astrocytoma,Supratentorial pilocytic astrocytoma,...,0.944328,supratentorial pilocytic astrocytoma,0.944301,Supratentorial pilocytic astrocytoma,0.944301,590.0,10006823160_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
18,71,9DC7BE79-C914-44C4-B89A-5C8442D45C86,83010,infratentoriell,14.0,36924,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,infratentorial pilocytic astrocytoma,Infratentorial pilocytic astrocytoma,...,0.999984,infratentorial pilocytic astrocytoma,0.999982,Infratentorial pilocytic astrocytoma,0.999982,982.0,101178130116_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176,6205,9215A2D7-E714-476C-B04A-FC1C68E585AA,69456,Links temporal,3.0,3713,Low-grade glial/glioneuronal/neuroepithelial t...,"diffuse glioma, MYB- or MYBL1-altered","diffuse glioma, MYB(L1)-altered, subtype A [an...","Angiocentric glioma, MYB/MYBL1-altered",...,0.999997,"diffuse glioma, MYB(L1)-altered, subtype A [an...",0.999994,"Angiocentric glioma, MYB/MYBL1-altered",0.999994,124325.0,9969477104_R01C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
2179,6208,B4C4055F-B3BC-4058-8697-7A2F3A0A2824,69462,Occipital,1.0,3715,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade ganglioglial/neuroepithelial tumour,desmoplastic infantile ganglioglioma/astrocytoma,Desmoplastic infantile ganglioglioma / desmopl...,...,1.000000,desmoplastic infantile ganglioglioma/astrocytoma,1.000000,Desmoplastic infantile ganglioglioma / desmopl...,1.000000,124327.0,9969477104_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
2184,6214,E4EF185C-E251-4F44-9CD3-A42DBF978994,69436,Temporalpol links,11.0,3704,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,Diffuse glioneuronal tumour with oligodendrogl...,Diffuse glioneuronal tumour with oligodendrogl...,...,0.998503,Diffuse glioneuronal tumour with oligodendrogl...,0.998408,Diffuse glioneuronal tumour with oligodendrogl...,0.998408,124334.0,9969477104_R06C01,No Chromothr

In [10]:
slide_annots.idat.uniq()

AttributeError: 'Series' object has no attribute 'uniq'

In [11]:
slide_meta

,Unnamed: 0,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,...,file_path,slide,prediction_superfamily,maxscore_superfamily,prediction_family,maxscore_family,prediction_class,maxscore_class,prediction_subclass,maxscore_subclass
idat,,,,,,,,,,,,,,,,,,,,,
10003886252_R05C02,1,6AAC7378-153F-4C5F-A4AC-430D127A802C,73872,"supratentoriell, temporal rechts",60.0,5397,Adult-type diffuse gliomas,"Glioblastoma, IDH-wildtype","glioblastoma, IDH-wildtype, RTK2 type","Glioblastoma, IDH-wildtype, RTK2 subtype",...,/omics/odcf/analysis/OE0606_projects/pancancer...,6AAC7378-153F-4C5F-A4AC-430D127A802C,Adult-type diffuse gliomas,0.995407,"Glioblastoma, IDH-wildtype",0.995217,"glioblastoma, IDH-wildtype, RTK2 type",0.794100,"Glioblastoma, IDH-wildtype, RTK2 subtype",0.794100
10003886253_R02C01,2,FE1BFC83-F163-4914-81C5-7605573D016F,73876,"spinal, C2 - C6",71.0,5396,Haematolymphoid tumours involving the CNS,B cell neoplasm,Primary diffuse large B cell lymphoma of the CNS,Primary diffuse large B cell lymphoma of the CNS,...,/omics/odcf/analysis/OE0606_projects/pancancer...,FE1BFC83-F163-4914-81C5-7605573D016F,Haematolymphoid tumours involving the CNS,0.940791,B cell neoplasm,0.940781,Primary diffuse large B cell lymphoma of the CNS,0.901420,Primary diffuse large B cell lymphoma of the CNS,0.901420
10003886253_R02C02,3,1EE2D6AD-2F2A-4457-BA86-4CC8D137249D,73896,"infratentoriell, sakral",30.0,4981,Ependymal tumours,myxopapillary ependymoma,myxopapillary ependymoma,Myxopapillary ependymoma,...,/omics/odcf/analysis/OE0606_projects/pancancer...,1EE2D6AD-2F2A-4457-BA86-4CC8D137249D,Ependymal tumours,0.999348,myxopapillary ependymoma,0.999087,myxopapillary ependymoma,0.999087,Myxopapillary ependymoma,0.999087
10003886253_R03C01,4,9625ECC7-5AA1-4ADF-B99E-B2F959A2317E,73878,"spinal, TH 11/12 extramedullär",71.0,5395,"Mesenchymal, non-meningothelial tumours involv...",Fibroblastic and myofibroblastic tumours,solitary fibrous tumour / haemangiopericytoma,Solitary fibrous tumour / haemangiopericytoma,...,/omics/odcf/analysis/OE0606_projects/pancancer...,9625ECC7-5AA1-4ADF-B99E-B2F959A2317E,"Mesenchymal, non-meningothelial tumours involv...",0.998369,Fibroblastic and myofibroblastic tumours,0.998369,solitary fibrous tumour / haemangiopericytoma,0.998369,Solitary fibrous tumour / haemangiopericytoma,0.998369
10003886253_R03C02,5,133E621F-1B3C-414D-9B39-4C3CFC77A928,73920,"supratentoriell, mittelliniennah, rechtsbetont...",70.0,5428,Adult-type diffuse gliomas,"Glioblastoma, IDH-wildtype","glioblastoma, IDH-wildtype, mesenchymal type","Glioblastoma, IDH-wildtype, [typical mesenchym...",...,/omics/odcf/analysis/OE0606_projects/pancancer...,133E621F-1B3C-414D-9B39-4C3CFC77A928,Adult-type diffuse gliomas,0.988475,"Glioblastoma, IDH-wildtype",0.987759,"glioblastoma, IDH-wildtype, mesenchymal type",0.487805,"Glioblastoma, IDH-wildtype, [typical mesenchym...",0.486671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9980102032_R03C01,6279,4C83D422-9DC5-4E3A-A120-6E2DEB47D553,71988,"infratentoriell, cerebellär links",27.0,4799,Medulloblastoma,"medulloblastoma, SHH-activated","medulloblastoma, SHH-activated, subtype 4","Medulloblastoma, SHH-activated, subtype 4",...,/omics/odcf/analysis/OE0606_projects/pancancer...,4C83D422-9DC5-4E3A-A120-6E2DEB47D553,Medulloblastoma,0.999928,"medulloblastoma, SHH-activated",0.999913,"medulloblastoma, SHH-activated, subtype 4",0.987410,"Medulloblastoma, SHH-activated, subtype 4",0.987410
9980102032_R04C01,6280,6936A1E9-D6D8-4A44-9CA7-989485943185,71990,"supratentoriell, Thalamus",10.0,4798,Control tissues,"control tissue, reactive tumour microenvironment","control tissue, reactive tumour microenvironment","Control tissue, reactive tumour microenvironment",...,/omics/odcf/analysis/OE0606_projects/pancancer...,6936A1E9-D6D8-4A44-9CA7-989485943185,Control tissues,0.885680,"control tissue, reactive tumour microenvironment",0.8855

In [12]:
path_to_extracted_features = '/home/p163v/histopathology/UKHD_Neuro/RetCLL_Features/'


patch_num = []
for x in slide_annots.uuid:
    fl = path_to_extracted_features+x+".h5"
    if(os.path.exists(fl)):
        patch_num.append(h5py.File(fl)['feats'].shape[0])
    else:
        patch_num.append(0)

In [13]:
slide_annots['patches'] = patch_num

In [14]:
slide_annots

,Unnamed: 0l,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,...,prediction_class,maxscore_class,prediction_subclass,maxscore_subclass,Unnamed: 0,idat,CT_class,Super Family,Oncotree code,patches
4,14,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,74022,"supratentoriell, frontal rechts",11.0,5459,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,dysembryoplastic neuroepithelial tumour,0.999795,Dysembryoplastic neuroepithelial tumour,0.999795,76.0,10003886259_R02C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,75
8,40,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,73136,frontal left,24.0,5311,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade",...,"diffuse glioma, IDH-mutant and 1p19q retained ...",0.947667,"Astrocytoma, IDH-mutant; high grade",0.947667,494.0,10006823123_R01C01,Chromothripsis,Adult-type diffuse gliomas,DIFG,0
10,47,F488896F-A788-4EAA-9B34-DB063CD29189,73178,"supratentoriell, temporal links",8.0,5342,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,dysembryoplastic neuroepithelial tumour,0.987715,Dysembryoplastic neuroepithelial tumour,0.987715,510.0,10006823130_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,998
15,66,B0250F69-2019-435C-8C95-A59B2A8CB70F,74226,"supratentoriell, frontal rechts",64.0,5520,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade ganglioglial/neuroepithelial tumour,supratentorial pilocytic astrocytoma,Supratentorial pilocytic astrocytoma,...,supratentorial pilocytic astrocytoma,0.944301,Supratentorial pilocytic astrocytoma,0.944301,590.0,10006823160_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,265
18,71,9DC7BE79-C914-44C4-B89A-5C8442D45C86,83010,infratentoriell,14.0,36924,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,infratentorial pilocytic astrocytoma,Infratentorial pilocytic astrocytoma,...,infratentorial pilocytic astrocytoma,0.999982,Infratentorial pilocytic astrocytoma,0.999982,982.0,101178130116_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,13197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176,6205,9215A2D7-E714-476C-B04A-FC1C68E585AA,69456,Links temporal,3.0,3713,Low-grade glial/glioneuronal/neuroepithelial t...,"diffuse glioma, MYB- or MYBL1-altered","diffuse glioma, MYB(L1)-altered, subtype A [an...","Angiocentric glioma, MYB/MYBL1-altered",...,"diffuse glioma, MYB(L1)-altered, subtype A [an...",0.999994,"Angiocentric glioma, MYB/MYBL1-altered",0.999994,124325.0,9969477104_R01C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,2146
2179,6208,B4C4055F-B3BC-4058-8697-7A2F3A0A2824,69462,Occipital,1.0,3715,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade ganglioglial/neuroepithelial tumour,desmoplastic infantile ganglioglioma/astrocytoma,Desmoplastic infantile ganglioglioma / desmopl...,...,desmoplastic infantile ganglioglioma/astrocytoma,1.000000,Desmoplastic infantile ganglioglioma / desmopl...,1.000000,124327.0,9969477104_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,6057
2184,6214,E4EF185C-E251-4F44-9CD3-A42DBF978994,69436,Temporalpol links,11.0,3704,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,Diffuse glioneuronal tumour with oligodendrogl...,Diffuse glioneuronal tumour with oligodendrogl...,...,Diffuse glioneuronal tumour with oligodendrogl...,0.998408,Diffuse glioneuronal tumour with oligodendrogl...,0.998408,124334.0,9969477104_R06C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial 

In [7]:
# np.random.seed(42)

# slide_train, slide_valid = train_test_split(np.array(slide_annots.uuid), train_size = 0.6)

# slide_valid, slide_test = train_test_split(slide_valid, train_size = 0.5)

In [15]:
slide_annots['labels'] = np.abs(1-slide_annots.CT_class.factorize(sort=True)[0])

In [16]:
np.random.seed(42)


kfold = StratifiedKFold(5)
kfold_test_splits = [x for x in kfold.split(np.array(slide_annots.uuid), np.array(slide_annots.labels))]




In [17]:
kfold_train_valid_test_splits = [train_test_split(x[0], train_size=0.75, stratify = np.array(slide_annots.labels)[x[0]]) + [x[1]] for x in kfold_test_splits]




In [18]:
for i in range(5):
    print('starting: '+str(i))
    os.makedirs('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/12062024_DIFG/'+str(i),exist_ok=True)
    slide_train = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][0]]
    slide_train_files = path_to_extracted_features + slide_train + '.h5'
    patch_num_train = np.array(slide_annots.patches)[kfold_train_valid_test_splits[i][0]]

    slide_valid = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][1]]
    slide_valid_files = path_to_extracted_features + slide_valid + '.h5'
    patch_num_valid = np.array(slide_annots.patches)[kfold_train_valid_test_splits[i][1]]

    slide_test = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][2]]
    slide_test_files = path_to_extracted_features + slide_test + '.h5'
    patch_num_test = np.array(slide_annots.patches)[kfold_train_valid_test_splits[i][2]]
    
    
    # patch_num_train = list()

    # for fl in slide_train_files:
    #     if(os.path.exists(fl)):
    #         patch_num_train.append(h5py.File(fl)['feats'].shape[0])
    #     else:
    #         patch_num_train.append(0)
    
    train_df = pd.DataFrame({'slide': slide_train,
                             'features': slide_train_files,
                             'patches': patch_num_train})
    train_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/12062024_DIFG/'+str(i)+'/train_set.csv')
    
#     patch_num_valid = list()

#     for fl in slide_valid_files:
#         if(os.path.exists(fl)):
#             patch_num_valid.append(h5py.File(fl)['feats'].shape[0])
#         else:
#             patch_num_valid.append(0)
    
    valid_df = pd.DataFrame({'slide': slide_valid,
                             'features': slide_valid_files,
                             'patches': patch_num_valid})
    valid_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/12062024_DIFG/'+str(i)+'/valid_set.csv')
    
#     patch_num_test = list()

#     for fl in slide_test_files:
#         if(os.path.exists(fl)):
#             patch_num_test.append(h5py.File(fl)['feats'].shape[0])
#         else:
#             patch_num_test.append(0)
    
    test_df = pd.DataFrame({'slide': slide_test,
                             'features': slide_test_files,
                             'patches': patch_num_test})
    test_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/12062024_DIFG/'+str(i)+'/test_set.csv')
    

starting: 0
starting: 1
starting: 2
starting: 3
starting: 4


In [19]:
[x.shape for x in kfold_train_valid_test_splits[0]]

[(375,), (126,), (126,)]